In [3]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from scipy.signal import argrelmin, argrelmax


In [41]:
ticker = "PLTR"

# Download 30-minute, 1-hour, and 1-day data for the past 60 days
thirty_min_data = yf.download(ticker, period="60d", interval="30m")
hourly_data = yf.download(ticker, period="60d", interval="1h")
daily_data = yf.download(ticker, period="60d", interval="1d")

# Reset index to ensure "Date" column is available
thirty_min_data.reset_index(inplace=True)
hourly_data.reset_index(inplace=True)
daily_data.reset_index(inplace=True)

daily_data
# daily_data.rename(columns={'Date': 'Datetime'}, inplace=True)

daily_data['Datetime'] = daily_data['Date'] + pd.to_timedelta('16:00:00')

c:\Users\edton\miniconda3\envs\trading\lib\site-packages\yfinance\scrapers\history.py:231: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [44]:
order = 5
max_idx = argrelmax(hourly_data["High"].values, order=order)[0]
min_idx = argrelmin(hourly_data["Low"].values, order=order)[0]

In [45]:
hourly_trace = go.Candlestick(x=hourly_data["Datetime"],
                              open=hourly_data["Open"],
                              high=hourly_data["High"],
                              low=hourly_data["Low"],
                              close=hourly_data["Close"],
                              name="1-Hour",
                              increasing_line_color="green", decreasing_line_color="red")

scatter_minima = go.Scatter(
            x=hourly_data["Datetime"][min_idx],
            y=hourly_data["Low"].iloc[min_idx],
            mode="markers+text",
            marker=dict(color="red", size=8),
            name="Min",
            textposition="bottom center",
        )

scatter_maxima = go.Scatter(
            x=hourly_data["Datetime"][max_idx],
            y=hourly_data["High"].iloc[max_idx],
            mode="markers+text",
            marker=dict(color="green", size=8),
            name="Max",
            textposition="bottom center",
        )


fig = go.Figure()

fig.add_trace(hourly_trace)
fig.add_trace(scatter_minima)
fig.add_trace(scatter_maxima)

# Customize the layout
fig.update_layout(
    xaxis=dict(rangebreaks=[dict(bounds=["sat", "mon"]), 
                            dict(bounds=[19, 6], pattern="hour")]),
    title=f"{ticker} Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    height=600,
    xaxis_rangeslider_visible=False,  # Disable the range slider for cleaner look
    hovermode="x unified",
)

# Show the figure
fig.show()

In [76]:
ticker = "pltr"

daily_data = yf.download(ticker, period="200d", interval="1d")




[*********************100%%**********************]  1 of 1 completed


In [77]:
daily_trace = go.Candlestick(x=daily_data.index,
                              open=daily_data["Open"],
                              high=daily_data["High"],
                              low=daily_data["Low"],
                              close=daily_data["Close"],
                              name="day candle",
                              increasing_line_color="green", decreasing_line_color="red")

fig = go.Figure()
fig.add_trace(daily_trace)





window = 5

daily_data['10D High'] = daily_data['Close'].rolling(window=window).max()
daily_data['New 10D High'] = daily_data['Close'] > daily_data['10D High'].shift(1)

daily_data['10D low'] = daily_data['Close'].rolling(window=window).min()
daily_data['New 10D low'] = daily_data['Close'] < daily_data['10D low'].shift(1)

new_data = daily_data[daily_data['New 10D High']]
day_max = go.Scatter(
            x=new_data.index,
            y=new_data["Close"],
            mode="markers+text",
            marker=dict(color="blue", size=8),
            name="max",
            textposition="bottom center",
        )
fig.add_trace(day_max)

new_data = daily_data[daily_data['New 10D low']]
day_min = go.Scatter(
            x=new_data.index,
            y=new_data["Close"],
            mode="markers+text",
            marker=dict(color="black", size=8),
            name="min",
            textposition="bottom center",
        )
fig.add_trace(day_min)




window = 10

daily_data['10D High'] = daily_data['Close'].rolling(window=window).max()
daily_data['New 10D High'] = daily_data['Close'] > daily_data['10D High'].shift(1)

daily_data['10D low'] = daily_data['Close'].rolling(window=window).min()
daily_data['New 10D low'] = daily_data['Close'] < daily_data['10D low'].shift(1)

new_data = daily_data[daily_data['New 10D High']]
day_max = go.Scatter(
            x=new_data.index,
            y=new_data["Close"] * 1.05,
            mode="markers+text",
            marker=dict(color="green", size=12),
            name="max",
            textposition="bottom center",
        )
fig.add_trace(day_max)

new_data = daily_data[daily_data['New 10D low']]
day_min = go.Scatter(
            x=new_data.index,
            y=new_data["Close"] * 0.95,
            mode="markers+text",
            marker=dict(color="grey", size=12),
            name="min",
            textposition="bottom center",
        )
fig.add_trace(day_min)







# Customize the layout
fig.update_layout(
    xaxis=dict(rangebreaks=[dict(bounds=["sat", "mon"])]),
    title=f"{ticker} Candlestick Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    height=600,
    xaxis_rangeslider_visible=False,  # Disable the range slider for cleaner look
    hovermode="x unified",
)

# Show the figure
fig.show()